# Preparação do ambiente

## Instalação de dependências

In [1]:
# !pip install pandas
# !pip install python-dotenv
# !pip install requests
# !pip install openai==0.28.1

## Importação das bibliotecas

In [2]:
# Importação das bibliotecas
import os
import openai
from dotenv import load_dotenv
import json

## Carregar variáveis de ambiente

In [3]:
load_dotenv()

True

## Configuração do OpenRouter

In [4]:
openai.api_key = os.getenv("OPENROUTER_API_KEY") # API Key do OpenRouter
openai.api_base = "https://openrouter.ai/api/v1" # URL da API do OpenRouter

In [5]:
# Modelo do OpenRouter
model_name = "meta-llama/llama-3.1-8b-instruct:free"

# Obter resposta do LLM

In [6]:
# Função para enviar uma pergunta e obter uma resposta do modelo
def get_chatbot_response(messages, system_prompt="", temperature=0, model=model_name):
    # Type check do messages
    if not isinstance(messages, list):
        raise TypeError("messages deve ser uma lista")

    # Lista de mensagens pro modelo
    input_messages = []
    for message in messages:
        # Adicionar dicionário à lista
        input_messages.append({
            "role": message["role"], # Quem está falando (usuário ou sistema)
            "content": message["content"] # Conteúdo da mensagem
        })

    response = openai.ChatCompletion.create(
        model=model, # Modelo de IA
        # Lista de mensagens
        messages=input_messages,
        # Temperatura: quantidade de aleatoriedade na resposta
        temperature=temperature, # 0 pois queremos resultados concretos
        top_p=0.8,
        max_tokens=2_000, # Limite de tokens na resposta
        # Token: unidade de medida do modelo de IA (palavras, sub-palavras, caracteres, ...)
    ).choices[0].message.content.strip()

    return response # Retornar resposta ao usuário

In [7]:
# Teste da função
msgs_usuario = [{"role": "user", "content": "Qual é a capital da Austrália?"}]
resposta = get_chatbot_response(msgs_usuario)
print(f"Usuário: {msgs_usuario[0]['content']}\nChatBot: {resposta}")

Usuário: Qual é a capital da Austrália?
ChatBot: Canberra.


# Engenharia de Prompt

## Structured output
Armazena a resposta do chatbot em um formato estruturado (como CSV ou JSON) para poder ser utilizado em outros sistemas/aplicações (como React Native).

### System Prompt
Prompt dado ao LLM que define seu comportamento.

In [8]:
system_prompt = """
Você é um assistente prestativo que responde perguntas sobre capitais de países.

Seu output deve ser um formato JSON estruturado exatamente como no exemplo abaixo. Você não tem permissão de escrever qualquer outra coisa além do objeto JSON.
[
    {
        "country": O país que o usuário perguntou a capital,
        "capital": A capital do país que o usuário perguntou,
    }
]
"""

system_prompt_message = {"role": "system", "content": system_prompt}

In [9]:
# Teste do system_prompt
messages = [
    system_prompt_message,
    {"role": "user", "content": "Qual é a capital da Nova Zelândia?"}]
resposta = get_chatbot_response(messages, system_prompt=system_prompt)
print(resposta)
print("Data type:", type(resposta))

# Converter resposta para JSON
json_resposta = json.loads(resposta)
print("\nJSON:", json_resposta)
print("Data type:", type(json_resposta))
print("Data type do objeto da lista:", type(json_resposta[0]))

# Retornar apenas a capital
print("\nCapital:", json_resposta[0]["capital"])

[
    {
        "country": "Nova Zelândia",
        "capital": "Wellington"
    }
]
Data type: <class 'str'>

JSON: [{'country': 'Nova Zelândia', 'capital': 'Wellington'}]
Data type: <class 'list'>
Data type do objeto da lista: <class 'dict'>

Capital: Wellington


In [10]:
# Teste 2
messages = [
    system_prompt_message,
    {"role": "user", "content": "Qual é a capital da França e do Brasil?"}]
resposta = json.loads(get_chatbot_response(messages, system_prompt=system_prompt))
print(resposta)
print("Data type:", type(resposta))
print("Data type do objeto da lista:", type(resposta[0]))

[{'country': 'França', 'capital': 'Paris'}, {'country': 'Brasil', 'capital': 'Brasília'}]
Data type: <class 'list'>
Data type do objeto da lista: <class 'dict'>


## Input Structuring

In [11]:
user_input = """
Obtenha para mim a capital dos seguintes países:
```
1. Itália
2. Alemanha
3. Canadá
4. Bélgica
```
"""

messages = [
    system_prompt_message,
    {"role": "user", "content": user_input}]
response = get_chatbot_response(messages)
print(json.loads(response))

[{'country': 'Itália', 'capital': 'Roma'}, {'country': 'Alemanha', 'capital': 'Berlim'}, {'country': 'Canadá', 'capital': 'Ottawa'}, {'country': 'Bélgica', 'capital': 'Bruxelas'}]


## Dê tempo ao modelo para pensar: Chain-of-thought (CoT)
Aumenta a previsão das respostas e reduz a chance de erros.

### Sem CoT:

In [25]:
# Exemplo: equação matemática
user_prompt = """
Calcule o resultado dessa equação: 259/1*8654+91072*33-12971

Seu output deve ser um JSON estruturado exatamente igual o exemplo abaixo. Você não tem permissão de escrever qualquer outra coisa além do objeto JSON (nem mesmo o passo a passo da resolução).
[{
    result: O número final resultado do cálculo da equação acima.

}]
"""
messages = [{"role": "user", "content": user_prompt}]
response = json.loads(get_chatbot_response(messages, system_prompt=system_prompt))
print(response)

{'result': 2161119}


In [28]:
resultado_real = 259/1*8654+91072*33-12971 # Resultado correto
print("Resultado correto:", resultado_real)
print("Margem de erro:", (response["result"] - resultado_real)) # Margem de erro

Resultado correto: 5233791.0
Margem de erro: -3072672.0


### Com CoT

In [ ]:
# Exemplo: equação matemática com etapas
user_prompt = """
Calcule o resultado dessa equação: 259/1*8654+91072*33-12971

Seu output deve ser um JSON estruturado exatamente igual o exemplo abaixo. Você não tem permissão de escrever qualquer outra coisa além do objeto JSON:
[{
    step: Aqui, você resolve a equação aos poucos, seguindo a ordem de operações BEDMAS. Você precisa mostrar seu trabalho e calcular cada etapa que leva ao resultado final. Sinta-se à vontade para escrever em texto livre.
    result: O número final resultado do cálculo da equação acima. O número deve ser um inteiro ou float.
}]
"""
messages = [{"role": "user", "content": user_prompt}]
response = "[" + get_chatbot_response(messages, system_prompt=system_prompt) + "]"

{
  "step": "Primeiramente, vamos resolver a multiplicação de 259 e 8654: 259*8654 = 2243516",
  "result": 2243516
}


In [57]:
print(response)

{'step': 'Primeiramente, vamos resolver a multiplicação de 259 e 8654: 259*8654 = 2243516', 'result': 2243516}


In [59]:
print("Resultado real:", resultado_real)
print("Resultado do modelo:", response["result"])
print("Margem de erro:", (response["result"] - resultado_real)) # Margem de erro

Resultado real: 5233791.0
Resultado do modelo: 2243516
Margem de erro: -2990275.0


# RAG